In [1]:
import os
import re
import time
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta


# ігнорування попередження SettingWithCopyWarning
pd.set_option('mode.chained_assignment', None)  


In [3]:
# конвертування строчки дати в формат дати 
d, m, y = [int(x) for x in df.at[0, "Date"].split("/")]
a = date(y, m, d)
a
# date(2023, 5, 9).isoweekday() == 2  # первірка дати на вівторок

NameError: name 'df' is not defined

In [2]:
list_columns =   [
        "Date",
        "HomeTeam",
        "AwayTeam",
        "rez_h_full",
        "rez_a_full",
        "rez_h_first",
        "rez_a_first",
        "h_first",
        "h_full",
        "a_first",
        "a_full",
    ] 
# funcs

def get_team_results(
                    h_team_name: str,
                    a_team_name: str,
                    matches_before: pd.DataFrame,
                    ) -> tuple[int, int, int, int] :
    """
    отримання результатів команд, останнього матчу
    повертає список з 4 чисел, різниці забитих мячів
      - дом. команди в першому таймі,
      - дом. команди в повному матчі,
      - гост. команди в першому таймі,
      - гост. команди в повному матчі,
    """
    h_team_match = matches_before[                            
                        (matches_before["HomeTeam"] == h_team_name) | 
                        (matches_before["AwayTeam"] == h_team_name)
                    ].tail(1) 
    
    a_team_match = matches_before[
                        (matches_before["HomeTeam"] == a_team_name) |
                        (matches_before["AwayTeam"] == a_team_name)
                    ].tail(1) 

    h_first = (
                    h_team_match.at[h_team_match.index[0], "HTHG"] - 
                    h_team_match.at[h_team_match.index[0], "HTAG"]
                )


    h_full = (
                    h_team_match.at[h_team_match.index[0], "FTHG"] - 
                    h_team_match.at[h_team_match.index[0], "FTAG"]
                )

    h_first = h_first if h_team_name in h_team_match["HomeTeam"].values else -h_first
    h_full = h_full if h_team_name in h_team_match["HomeTeam"].values else -h_full


    a_first = (
                    a_team_match.at[a_team_match.index[0], "HTHG"] - 
                    a_team_match.at[a_team_match.index[0], "HTAG"]
                )

    a_full = (
                    a_team_match.at[a_team_match.index[0], "FTHG"] - 
                    a_team_match.at[a_team_match.index[0], "FTAG"]
                )
    a_first = a_first if a_team_name in a_team_match["HomeTeam"].values else -a_first
    a_full = a_full if a_team_name in a_team_match["HomeTeam"].values else -a_full
    
    return (
        h_first,
        h_full,
        a_first,
        a_full,
    )


def get_rez_data_df(df_data):
    rez_data = []
    for ind in df_data.index:
        current_match = df_data.loc[ind]
        matches_before = df_data.loc[:ind-1]
        h_team_name = current_match["HomeTeam"]
        a_team_name = current_match["AwayTeam"]
        if ( 
            (h_team_name in matches_before["HomeTeam"].values or
                     h_team_name in matches_before["AwayTeam"].values) and 
            (a_team_name in matches_before["HomeTeam"].values or
                     a_team_name in matches_before["AwayTeam"].values)  
        ):
            h_first, h_full, a_first, a_full = get_team_results(
                                                         h_team_name,
                                                         a_team_name,
                                                        matches_before
                                                        )
            """
            print(ind, h_team_name, a_team_name)
            print()
            print(matches_before.tail(15))
            print()

            print(h_full, h_first)

            print()
            print(a_full, a_first)
            """

            data_row = [
                *current_match.values,
                h_first,
                h_full,
                a_first,
                a_full
            ]
            rez_data.append(data_row)
    return rez_data


def get_clean_data_df(df):
    df.loc[:, "Date"] =  [date.fromisoformat("-".join(x.split("/")[::-1]))  for x in df.loc[:, "Date"].values]

    df_data = df.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HTHG", "HTAG"]]

    clean_data = pd.DataFrame(
                            data=get_rez_data_df(df_data),
                            columns=list_columns
                )
    return clean_data

In [342]:
def get_total_work(clean_data, tuesdays):

    rez_dict = {
        "win" : 0,
        "lose" : 0
    }
    
    for tuesday in tuesdays:

        work_df = clean_data[
            (clean_data["Date"] >= tuesday - timedelta(days=14)) &
            (clean_data["Date"] < tuesday) 
        ] 


        target_df = clean_data[
            (clean_data["Date"] >= tuesday)  &
            (clean_data["Date"] < tuesday + timedelta(days=7)) 
        ]

        # минулі результати
        work_df["index"] = work_df.index
        work_df_np = work_df[[
                                "h_first", "h_full",
                                "a_first", "a_full",
                                "rez_h_full", "rez_a_full", "rez_h_first", "rez_a_first",
                                "index"
                    ]].to_numpy()


        # майбутні результати
        target_df["index"] = target_df.index
        target_df_np = target_df[[
                                "h_first", "h_full",
                                "a_first", "a_full",
                                "rez_h_full", "rez_a_full", "rez_h_first", "rez_a_first",
                                "index"
                    ]].to_numpy()


#         if filtred_work_df_np.shape[0] == 0: continue

        for data in target_df_np:
            h_first = data[0]
            h_full = data[1]
            a_first = data[2]
            a_full = data[3]

            filtred_work_df_np =  work_df_np[
                    (work_df_np[:, 0] == h_first)
#                     &
#                     (work_df_np[:, 1] == h_full)
                    &
                    (work_df_np[:, 2] == a_first)
#                     &(work_df_np[:, 3] == a_full)
                ]
            if not filtred_work_df_np.shape[0] >= 2: continue
            win_df_np_shape = filtred_work_df_np[
#                     filtred_work_df_np[:, 4] == filtred_work_df_np[:, 5]
    #                 filtred_work_df_np[:, 4] <= filtred_work_df_np[:, 5]
    #                 filtred_work_df_np[:, 4] +  filtred_work_df_np[:, 5] < 3.5
#                     (filtred_work_df_np[:, 4] > 0.5) & (filtred_work_df_np[:, 5] > 0.5)

    #                 first time
                    filtred_work_df_np[:, 6] == filtred_work_df_np[:, 7]
    #                 filtred_work_df_np[:, 6] != filtred_work_df_np[:, 7]
#                     filtred_work_df_np[:,6] + filtred_work_df_np[:, 7] < 0.5
#                     filtred_work_df_np[:,6] + filtred_work_df_np[:, 7] > 1.5
                ].shape[0]
            lose_df_np_shape = filtred_work_df_np.shape[0] - win_df_np_shape
            
            if not win_df_np_shape > lose_df_np_shape: continue
#             if not lose_df_np_shape == 0: continue
            """
            print("="*25 )

            print(work_df_np)
            print( )
            print(filtred_work_df_np)
            print( )
            print( win_df_np_shape, lose_df_np_shape)
            print( )
            print(data )
            print( )
            raise
            """
#             if data[4] == data[5]:
#             if data[4] <= data[5]:
#             if data[4] + data[5] < 3.5:
#             if data[4] > 0.5 and data[5] > 0.5:
                
            if data[6] == data[7]:
#             if data[6] != data[7]:
#             if data[6] + data[7] < 0.5:
#             if data[6] + data[7] > 1.5:
                rez_dict["win"] += 1
            else:
                rez_dict["lose"] += 1

    return rez_dict    

In [343]:
total_rez_dict = {
    "win" : 0,
    "lose" : 0
}
for file_name in os.listdir("./old_data"):
# for file_name in ["sp1_21_22.csv"]:
    df = pd.read_csv(f"./old_data/{file_name}")
#     df = pd.read_csv("./data/n1_21_22.csv")


    clean_data = get_clean_data_df(df)

    start_day = clean_data["Date"].values[0]
    end_day = clean_data["Date"].values[-1]

    list_days = []
    while start_day <= end_day:
        list_days.append(start_day)
        start_day += timedelta(days=1)

    only_tuesdays = list(
                        filter(
                            lambda day: day.isoweekday() == 2,
                            list_days
                        )
                    )
    rez_dict = get_total_work(clean_data, only_tuesdays)
    win = rez_dict["win"]
    total_rez_dict["win"] += win
    
    lose = rez_dict["lose"]
    total_rez_dict["lose"] += lose
    
    proc = round(win/(win + lose), 3) if (win + lose) > 0 else 0
    print(
        file_name, win, lose, proc
    )
print(total_rez_dict, round(total_rez_dict["win"]/(total_rez_dict["win"]+total_rez_dict["lose"]), 3))
print(total_rez_dict["win"]*2.3 - total_rez_dict["win"] - total_rez_dict["lose"])

some_test.csv 0 0 0
e0_21_22.csv 5 5 0.5
sp1_21_22.csv 6 21 0.222
t1_18_19.csv 6 14 0.3
i2_21_22.csv 15 20 0.429
e0_19_20.csv 14 20 0.412
p1_18_19.csv 11 12 0.478
e2_21_22.csv 35 32 0.522
e1_21_22.csv 21 30 0.412
e0_20_21.csv 7 9 0.438
p1_21_22.csv 12 13 0.48
f1_21_22.csv 13 16 0.448
t1_21_22.csv 16 20 0.444
i1_21_22.csv 6 9 0.4
n1_21_22.csv 5 10 0.333
n1_18_19.csv 1 3 0.25
sp2_19_20.csv 47 39 0.547
{'win': 220, 'lose': 273} 0.446
12.999999999999943


In [60]:
нічья перший тайм, перемог більше ніж поразок
 
====

h_first+h_full+a_first, при кількості співпадінь 2 - {'win': 47, 'lose': 61} 0.435 => 0
h_first+h_full+a_full, при кількості співпадінь 2 - {'win': 29, 'lose': 36} 0.446 => 1.69
h_first+h_full, при кількості співпадінь 2 - {'win': 211, 'lose': 268} 0.441 => 6.29

h_first+a_first+a_full, при кількості співпадінь 2 - {'win': 53, 'lose': 60} 0.469 => 8.8
h_full+a_first+a_full, при кількості співпадінь 2 - {'win': 29, 'lose': 36} 0.446 => 1.6
a_first+a_full, при кількості співпадінь 2 - {'win': 197, 'lose': 255} 0.436 => 1.0

h_first+a_first, при кількості співпадінь 2 - {'win': 220, 'lose': 273} 0.446 => 12.9
h_full+a_full , - {'win': 96, 'lose': 127} 0.43 => -2.2

====
нічья перший тайм,  поразок немає
 
====
a_first+a_full, при кількості співпадінь 2 - {'win': 87, 'lose': 99} 0.468 => 14.09

h_first+h_full, при кількості співпадінь 2 - - {'win': 81, 'lose': 97} 0.455 => 8.29

SyntaxError: invalid syntax (3468536097.py, line 1)

In [282]:
total_rez_dict["win"]*3 - total_rez_dict["win"] - total_rez_dict["lose"]



-8

In [283]:
total_rez_dict["lose"]*1.4 - total_rez_dict["win"] - total_rez_dict["lose"]

-4.400000000000006